In [1]:
import xlrd 
import pubchempy as pcp
import pickle

import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem import PandasTools
from rdkit.Chem.Draw import IPythonConsole
import matplotlib.pyplot as plt
import seaborn as sns

from mol2vec.features import mol2alt_sentence, MolSentence, DfVec, sentences2vec, mol2sentence
from mol2vec.helpers import depict_identifier, plot_2D_vectors, IdentifierTable, mol_to_svg

In [2]:
pickle_in = open("fold-0.pickle","rb")
load_dict =  pickle.load(pickle_in)

data_frame = load_dict['shuffled']

data_frame

,No.,FileHandle,SoluteName,Formula,Subset,Charge,Level1,Level2,Level3,Solvent,...,HP,OP,S,HS,OS,SP,SS,TotalArea,SoluteSMILES,SolventSMILES
1758,1759,0438pho,"diethyl2,4-dichlorophenylthiophosphate",H13C10O3P1S1CL2,[g],0,11,4,0,octanol,...,0.000,16.033,34.582,0.0,0.0,24.737,0.0,331.933,C1C2C(C(C1Cl)Cl)C3(C(=C(C2(C3(Cl)Cl)Cl)Cl)Cl)Cl,CCCCCCCCO
788,789,0071proa,propanal,H6C3O1,[g],0,2,4,1,diethylether,...,0.000,0.000,0.000,0.0,0.0,0.000,0.0,124.289,CCC=O,CCOCC
1726,1727,0210dic,"1,1-dichloro-2,2-difluoroethylmethylether",H4C3O1F2CL2,[g],0,9,1,0,octanol,...,0.000,0.000,0.000,0.0,0.0,0.000,0.0,178.993,CC1=CC(=O)CC(C1)(C)C,CCCCCCCCO
1939,1940,0110but,butylamine,H11C4N1,[g],0,3,1,0,secbutanol,...,0.000,0.000,0.000,0.0,0.0,0.000,0.0,164.243,CCCCN,CCC(C)O
1467,1468,0008noc,n-octane,H18C8,[g],0,2,1,1,methylformamide,...,0.000,0.000,0.000,0.0,0.0,0.000,0.0,243.846,CCCCCCCC,CNC=O
2110,2111,0021eth,ethene,H4C2,[a],0,2,1,4,water,...,0.000,0.000,0.000,0.0,0.0,0.000,0.0,87.417,C=C,O
739,740,0087pro,propanoicacid,H6C3O2,[g],0,2,5,0,dichloroethane,...,0.000,0.000,0.000,0.0,0.0,0.000,0.0,135.630,CCC(=O)O,CC(Cl)Cl
1196,1197,0172Edi,"E-1,2-dichloroethene",H2C2CL2,[g],0,6,2,0,hexadecane,...,0.000,0.000,0.000,0.0,0.0,0.000,0.0,126.855,C(=CCl)Cl,CCCCCCCCCCCCCCCC
111,112,0150mhy,m-hydroxybenzaldehyde,H6C7O2,[g],0,2,9,0,benzene,...,0.000,0.000,0.000,0.0,0.0,0.000,0.0,179.066,C1=CC(=CC(=C1)O)C=O,C1=CC=CC=C1
1820,1821,test4002,diiodomethane,H2C1I2,[g],0,15,0,0,octanol,...,0.000,0.000,0.000,0.0,0.0,0.000,0.0,136.531,C(I)I,CCCCCCCCO


In [3]:
# loc = ("MNSol_alldata.xlsx") 
# wb = xlrd.open_workbook(loc) 
# sheet = wb.sheet_by_index(0) 

# solutes = sheet.col_values(2)
# solvents = sheet.col_values(9)
# energies = sheet.col_values(10)

# solute_smis = []
# solvent_smis = []

# for solute in solutes[100:110]:
#     for compound in pcp.get_compounds(solute, 'name'):
#         solute_smis.append(compound.isomeric_smiles)
        
# for solvent in solvents[100:110]:
#     for compound in pcp.get_compounds(solvent, 'name'):
#         solvent_smis.append(compound.isomeric_smiles)

solute_smis = data_frame['SoluteSMILES'].tolist()
solvent_smis = data_frame['SolventSMILES'].tolist()
energies = data_frame['DeltaGsolv'].tolist()

print(len(solute_smis))
print(len(solvent_smis))
print(len(energies))

2530
2530
2530


In [4]:
solutesmis = [Chem.MolFromSmiles(x) for x in solute_smis]
solventsmis = [Chem.MolFromSmiles(x) for x in solvent_smis]

RDKit WARNING: [14:44:57] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [14:44:57] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [14:44:57] WARNING: not removing hydrogen atom without neighbors


In [5]:
solute_sentences = []
solvent_sentences = []

for solute_smi in solutesmis:
    solute_sentences.append(mol2alt_sentence(solute_smi, 1))
    
for solvent_smi in solventsmis:
    solvent_sentences.append(mol2alt_sentence(solvent_smi, 1))
    
print(solute_sentences[6])
print(solvent_sentences[6])

['2246728737', '3542456614', '2245384272', '2064788354', '2246699815', '4278941385', '864942730', '1510328189', '864662311', '1533864325']
['2246728737', '3537119515', '2245273601', '2406620359', '1016841875', '170898322', '1016841875', '170898322']


In [13]:
from gensim.models import word2vec
model = word2vec.Word2Vec.load('model_300dim.pkl')

solute_sequences = []
count = 0
solvent_sequences = []
count1=0
l = len(solute_smis)

for i in range(l):
    flag = 0
    solute_substructures = []
    for identifier in solute_sentences[i]:
        try:
            solute_substructures.append(model.wv.word_vec(identifier))
        except:
            flag = 1
            continue
    
    solvent_substructures = []
    for identifier in solvent_sentences[i]:
        try:
            solvent_substructures.append(model.wv.word_vec(identifier))
        except:
            flag = 1
            continue
            
    if flag == 1:
        count += 1
        continue
    solute_sequences.append(solute_substructures)
    solvent_sequences.append(solvent_substructures)
    
    
print(count)
# solute_mol2vecs = [DfVec(x) for x in sentences2vec(solute_sentences, model, unseen='UNK')]
# solvent_mol2vecs = [DfVec(x) for x in sentences2vec(solvent_sentences, model, unseen='UNK')]

401


In [37]:
for i in solute_sequences:
    print(np.asarray(i).shape)
print('\n')
for i in solvent_sequences:
    print(np.asarray(i).shape)

(6, 300)
(8, 300)
(8, 300)
(10, 300)
(14, 300)
(12, 300)
(14, 300)
(14, 300)
(14, 300)
(16, 300)


(12, 300)
(12, 300)
(12, 300)
(12, 300)
(12, 300)
(12, 300)
(12, 300)
(12, 300)
(12, 300)
(12, 300)


In [38]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable

In [39]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layer, output_dim, batch_size, bidirectional = True):
        super(LSTMModel, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.num_layer = num_layer
        self.bidirectional = bidirectional
        self.batch_size = batch_size

        self.solv_lstm = nn.LSTM(input_dim, hidden_dim, num_layer, batch_first=True, bidirectional = bidirectional)
        self.solu_lstm = nn.LSTM(input_dim, hidden_dim, num_layer, batch_first=True, bidirectional = bidirectional)

        self.fc1 = nn.Linear(hidden_dim*2*2, 2000)
        self.fc2 = nn.Linear(2000, output_dim)
        
    def forward(self, inputs_solv, inputs_solu):
        # Initialize hidden state
        h0_solv = torch.zeros(self.num_layer*(1 + int(self.bidirectional)), self.batch_size, self.hidden_dim)
        h0_solu = torch.zeros(self.num_layer*(1 + int(self.bidirectional)), self.batch_size, self.hidden_dim)

        # Initialize cell state
        c0_solv = torch.zeros(self.num_layer*(1 + int(self.bidirectional)), self.batch_size, self.hidden_dim)
        c0_solu = torch.zeros(self.num_layer*(1 + int(self.bidirectional)), self.batch_size, self.hidden_dim)
        
        H, _ = self.solv_lstm(inputs_solv, (h0_solv, c0_solv)) # out: tensor of shape (batch_size, seq_length, hidden_size*2)
        G, _ = self.solu_lstm(inputs_solu, (h0_solu, c0_solu))
        
        # shared attention layer
        Hsize = H.size(1)
        Gsize = G.size(1)
        a_score = torch.zeros(Hsize, Gsize)
        
        G = torch.squeeze(G)
        H = torch.squeeze(H)
        
        for i in range(Hsize):
            for j in range(Gsize):
                a_score[i][j] = torch.matmul(H[i],torch.t(G[j]))
        
        a = F.softmax(a_score,1)
        
        P = torch.matmul(a, G)
        Q = torch.matmul(torch.t(a), H)
        
        # maxpooling layer
        u = torch.max(H, P)
        v = torch.max(G, Q)
        
        inpu = torch.sum(u, dim=0)
        inpv = torch.sum(v, dim=0)
        
        inp = torch.cat((inpu, inpv), 0)
        
        # mlp
        x = F.relu(self.fc1(inp)) #number of layers? concat? sum?
        solvE = self.fc2(x)
      
        return solvE


In [45]:
bidirectional = True

input_dim = 300
hidden_dim = 150
layer_dim = 1
output_dim = 1

batch_size = 1

In [46]:
model = LSTMModel(input_dim, hidden_dim, layer_dim, output_dim, batch_size)

# PRINTING MODEL & PARAMETERS 
print(model)
print(len(list(model.parameters())))
for i in range(len(list(model.parameters()))):
    print(list(model.parameters())[i].size())
    
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

LSTMModel(
  (solv_lstm): LSTM(300, 150, batch_first=True, bidirectional=True)
  (solu_lstm): LSTM(300, 150, batch_first=True, bidirectional=True)
  (fc1): Linear(in_features=600, out_features=2000, bias=True)
  (fc2): Linear(in_features=2000, out_features=1, bias=True)
)
20
torch.Size([600, 300])
torch.Size([600, 150])
torch.Size([600])
torch.Size([600])
torch.Size([600, 300])
torch.Size([600, 150])
torch.Size([600])
torch.Size([600])
torch.Size([600, 300])
torch.Size([600, 150])
torch.Size([600])
torch.Size([600])
torch.Size([600, 300])
torch.Size([600, 150])
torch.Size([600])
torch.Size([600])
torch.Size([2000, 600])
torch.Size([2000])
torch.Size([1, 2000])
torch.Size([1])
The model has 2,288,801 trainable parameters


In [52]:
learning_rate = 0.0002
nest_momentum = 0.9

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=nest_momentum, nesterov=True)

In [53]:
num_epochs = 10

# Train the model
total_step = len(solute_sequences)

for epoch in range(num_epochs):
    for i in range(total_step-5):
        solute = solute_sequences[i]
        solvent = solvent_sequences[i]
        energy = torch.FloatTensor([energies[i+100]])
        
        # Forward pass
        output = model((torch.FloatTensor([solvent])), (torch.FloatTensor([solute])))
        loss = criterion(output, energy)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/10], Step [1/10], Loss: 10.3223
Epoch [1/10], Step [2/10], Loss: 13.5334
Epoch [1/10], Step [3/10], Loss: 5.1306
Epoch [1/10], Step [4/10], Loss: 7.9733
Epoch [1/10], Step [5/10], Loss: 7.8331
Epoch [2/10], Step [1/10], Loss: 0.1300
Epoch [2/10], Step [2/10], Loss: 0.0012
Epoch [2/10], Step [3/10], Loss: 1.2592
Epoch [2/10], Step [4/10], Loss: 0.6548
Epoch [2/10], Step [5/10], Loss: 0.7951
Epoch [3/10], Step [1/10], Loss: 2.3461
Epoch [3/10], Step [2/10], Loss: 2.2051
Epoch [3/10], Step [3/10], Loss: 2.4845
Epoch [3/10], Step [4/10], Loss: 0.4743
Epoch [3/10], Step [5/10], Loss: 0.0277
Epoch [4/10], Step [1/10], Loss: 0.3133
Epoch [4/10], Step [2/10], Loss: 0.0709
Epoch [4/10], Step [3/10], Loss: 0.1306
Epoch [4/10], Step [4/10], Loss: 0.1848
Epoch [4/10], Step [5/10], Loss: 0.7952
Epoch [5/10], Step [1/10], Loss: 0.0247
Epoch [5/10], Step [2/10], Loss: 0.0878
Epoch [5/10], Step [3/10], Loss: 0.0037
Epoch [5/10], Step [4/10], Loss: 0.3444
Epoch [5/10], Step [5/10], Loss: 0.701

In [55]:
# Test the model
outputs = []

with torch.no_grad():
    for i in range(total_step-5):
        solute = solute_sequences[i+5]
        solvent = solvent_sequences[i+5]
        energy = torch.FloatTensor([energies[i+105]])
        
        output = model((torch.FloatTensor([solvent])), (torch.FloatTensor([solute])))
        outputs.append(output)

    print('MSE: {}'.format(criterion(torch.FloatTensor(energies[105:110]).squeeze(), torch.FloatTensor([outputs])))) 

MSE: 7.441834926605225


In [61]:
print(outputs)
print(energies[105:110])

[tensor([-3.1967]), tensor([-3.5724]), tensor([-3.4067]), tensor([-3.3865]), tensor([-3.7676])]
[-5.28, -6.88, -5.86, -6.17, -6.64]


In [ ]:
# Save the model checkpoint
#torch.save(model.state_dict(), 'model.ckpt')